In [1]:
import os
import unittest
from logger import setup_logger
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph
from load_cfg import OPENAI_API_KEY, LANGCHAIN_API_KEY, WORKING_DIRECTORY

class TestMultiAgentSystem(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        # Set environment variables
        os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
        os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
        os.environ["LANGCHAIN_TRACING_V2"] = "true"
        os.environ["LANGCHAIN_PROJECT"] = "Multi-Agent Data Analysis System Test"
        
        # Set up logger
        cls.logger = setup_logger()
        
        # Initialize test LLMs
        try:
            cls.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
            cls.power_llm = ChatOpenAI(model="gpt-4o", temperature=0.5)
            cls.json_llm = ChatOpenAI(
                model="gpt-4o",
                model_kwargs={"response_format": {"type": "json_object"}},
                temperature=0
            )
            cls.logger.info("Test language models initialized successfully.")
        except Exception as e:
            cls.logger.error(f"Error initializing test language models: {str(e)}")
            raise

    def test_agent_creation(self):
        """Test creation of various agents"""
        from create_agent import create_agent, create_supervisor
        
        # Test hypothesis agent creation
        hypothesis_agent = create_agent(
            self.llm,
            [],  # Empty tools list for testing
            "Test prompt",
            ["Test"],
            WORKING_DIRECTORY
        )
        self.assertIsNotNone(hypothesis_agent)
        
        # Test supervisor creation
        supervisor = create_supervisor(
            self.power_llm,
            "Test prompt",
            ["Test"]
        )
        self.assertIsNotNone(supervisor)

    def test_workflow_creation(self):
        """Test workflow graph creation"""
        from state import State
        
        workflow = StateGraph(State)
        self.assertIsNotNone(workflow)
        
        # Test basic node addition
        workflow.add_node("TestNode", lambda state: state)
        
    def test_tools(self):
        """Test tool initialization"""
        from tools.internet import google_search, scrape_webpages_with_fallback
        from tools.basetool import execute_code, execute_command
        from tools.FileEdit import create_document, read_document
        
        # Test tool availability
        self.assertTrue(callable(google_search))
        self.assertTrue(callable(scrape_webpages_with_fallback))
        self.assertTrue(callable(execute_code))
        self.assertTrue(callable(execute_command))
        self.assertTrue(callable(create_document))
        self.assertTrue(callable(read_document))

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


ModuleNotFoundError: No module named 'logger'